In [1]:
import pandas as pd

features = ['accommodates','bedrooms','bathrooms','beds','price','minimum_nights','maximum_nights','number_of_reviews']

dc_listings = pd.read_csv('listings.csv')

dc_listings = dc_listings[features]
print(dc_listings.shape)

dc_listings.head()

(3723, 8)


,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,4,1.0,1.0,2.0,$160.00,1,1125,0
1,6,3.0,3.0,3.0,$350.00,2,30,65
2,1,1.0,2.0,1.0,$50.00,2,1125,1
3,2,1.0,1.0,1.0,$95.00,1,1125,0
4,4,1.0,1.0,1.0,$50.00,7,1125,0


In [2]:
import numpy as np

our_acc_value = 3

dc_listings['distance'] = np.abs(dc_listings.accommodates - our_acc_value)
dc_listings.distance.value_counts().sort_index()

0      461
1     2294
2      503
3      279
4       35
5       73
6       17
7       22
8        7
9       12
10       2
11       4
12       6
13       8
Name: distance, dtype: int64

In [3]:
dc_listings = dc_listings.sample(frac=1,random_state=0)
dc_listings = dc_listings.sort_values('distance')
dc_listings.price.head()

2645     $75.00
2825    $120.00
2145     $90.00
2541     $50.00
3349    $105.00
Name: price, dtype: object

In [4]:
dc_listings['price'] = dc_listings.price.str.replace('\$|,','').astype(float)
mean_price = dc_listings.price.iloc[:5].mean()
mean_price

88.0

In [5]:
dc_listings.tail()

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews,distance
763,16,1.0,1.0,1.0,1000.0,1,1125,0,13
2560,16,1.0,1.0,2.0,60.0,3,60,0,13
1594,16,10.0,8.0,13.0,1250.0,3,1125,0,13
1224,16,1.0,2.0,1.0,499.0,1,1125,0,13
1596,16,5.0,3.5,5.0,299.0,3,365,8,13


In [6]:
dc_listings.size

33507

In [7]:
dc_listings.shape

(3723, 9)

In [8]:
dc_listings.drop('distance',axis=1)
train_df = dc_listings.copy().iloc[:2792]
test_df = dc_listings.copy().iloc[2792:]

In [9]:
def predict_price(new_listing_value, feature_column):
    temp_df = train_df
    temp_df['distance'] = np.abs(dc_listings[feature_column] - new_listing_value)
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predict_price = knn_5.mean()
    return predict_price

In [10]:
test_df['predicted_price'] = test_df.accommodates.apply(predict_price, feature_column='accommodates')
test_df.head()

,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews,distance,predicted_price
2850,1,1.0,1.0,1.0,40.0,2,1125,105,2,83.6
2279,1,1.0,1.0,1.0,45.0,1,1125,8,2,83.6
2771,5,2.0,2.0,3.0,217.0,3,730,19,2,340.4
910,5,2.0,2.0,3.0,415.0,1,1125,1,2,340.4
2434,5,1.0,1.0,3.0,275.0,3,1125,9,2,340.4


In [11]:
test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**2
mse = test_df['squared_error'].mean()
rmse = mse**(1/2)
rmse

212.9892796705153

In [12]:
for feature in ['accommodates','bedrooms','bathrooms','number_of_reviews']:
    test_df['predicted_price'] = test_df[feature].apply(predict_price, feature_column=feature)
    test_df['squared_error'] = (test_df['predicted_price'] - test_df['price'])**2
    mse = test_df['squared_error'].mean()
    rmse = mse**(1/2)
    print('RMSE for the {} column: {}'.format(feature, rmse))

RMSE for the accommodates column: 212.9892796705153
RMSE for the bedrooms column: 199.80935328065047
RMSE for the bathrooms column: 230.2471670568422
RMSE for the number_of_reviews column: 235.91327066995507


In [13]:
from sklearn.preprocessing import StandardScaler
features = ['accommodates','bedrooms','bathrooms','beds','price','minimum_nights','maximum_nights','number_of_reviews']
dc_listings = pd.read_csv('listings.csv')
dc_listings = dc_listings[features]
dc_listings['price'] = dc_listings.price.str.replace('\$|,', '').astype(float)
dc_listings = dc_listings.dropna()
dc_listings[features] = StandardScaler().fit_transform(dc_listings[features])
normalized_listings = dc_listings
print(dc_listings.shape)
normalized_listings.head()

(3671, 8)


,accommodates,bedrooms,bathrooms,beds,price,minimum_nights,maximum_nights,number_of_reviews
0,0.401420,-0.249501,-0.439211,0.297386,0.081119,-0.341421,-0.016575,-0.516779
1,1.399466,2.129508,2.969551,1.141704,1.462622,-0.065047,-0.016606,1.706767
2,-1.095648,-0.249501,1.265170,-0.546933,-0.718699,-0.065047,-0.016575,-0.482571
3,-0.596625,-0.249501,-0.439211,-0.546933,-0.391501,-0.341421,-0.016575,-0.516779
4,0.401420,-0.249501,-0.439211,-0.546933,-0.718699,1.316824,-0.016575,-0.516779


In [14]:
norm_train_df = normalized_listings.copy().iloc[:2792]
norm_test_df = normalized_listings.copy().iloc[2792:]
norm_train_df.shape

(2792, 8)

In [15]:
from scipy.spatial import distance
first_listing = normalized_listings.iloc[0][['accommodates','bedrooms']]
twenty_listing = normalized_listings.iloc[20][['accommodates','bedrooms']]
first_twenty_distance = distance.euclidean(first_listing, twenty_listing)
first_twenty_distance

3.8698195887683675

In [34]:
def predict_price_multivariate(new_listing_value, feature_columns):
    temp_df = norm_train_df
    temp_df['distance'] = distance.cdist(temp_df[feature_columns], [new_listing_value[feature_columns]], metric='euclidean')
    temp_df = temp_df.sort_values('distance')
    knn_5 = temp_df.price.iloc[:5]
    predicted_price = knn_5.mean()
    return predicted_price

In [35]:
cols = ['accommodates', 'bathrooms']
norm_test_df['predicted_price'] = norm_test_df[cols].apply(predict_price_multivariate,feature_columns=cols,axis=1)    
norm_test_df['squared_error'] = (norm_test_df['predicted_price'] - norm_test_df['price'])**(2)
mse = norm_test_df['squared_error'].mean()
rmse = mse ** (1/2)
print(rmse)

0.7894063922577531


In [52]:
from sklearn.neighbors import KNeighborsRegressor
cols = ['accommodates', 'bathrooms']
knn = KNeighborsRegressor(10)
knn.fit(norm_train_df[cols], norm_train_df['price'])
two_features_predictions = knn.predict(norm_test_df[cols])

In [53]:
from sklearn.metrics import mean_squared_error
two_feature_mse = mean_squared_error(norm_test_df['price'], two_features_predictions)
two_feature_rmse = two_feature_mse ** (1/2)
print(two_feature_rmse)

0.8242227261342391


In [59]:
cols = ['accommodates','bedrooms','bathrooms','beds','minimum_nights','maximum_nights','number_of_reviews']
knn = KNeighborsRegressor(9)
knn.fit(norm_train_df[cols], norm_train_df['price'])
multi_features_predictions = knn.predict(norm_test_df[cols])
multi_feature_mse = mean_squared_error(norm_test_df['price'], multi_features_predictions)
multi_feature_rmse = multi_feature_mse ** (1/2)
print(multi_feature_rmse)

0.8001050282021709
